We will perform 'Sentiment analysis' using Word2Vec and AvgWord2Vec

Importing libraries

In [1]:
import pandas as pd  
import numpy as np

In [2]:
##Reading the dataset 
df = pd.read_csv('IMDB Dataset(CSV)') 
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [3]:
##Finding basic info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [4]:
df.shape

(50000, 2)

In [5]:
##Since we are using it for practise purpose , we can provide less number of rows  
df = df.iloc[:10000]  
df.shape

(10000, 2)

Basic EDA

In [6]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [7]:
df.duplicated().sum()

17

In [8]:
df.drop_duplicates(inplace = True)

In [9]:
df.duplicated().sum()   
##We have deleted all the duplicates

0

In [10]:
df.columns

Index(['review', 'sentiment'], dtype='object')

In [11]:
df['sentiment'].value_counts(normalize=True)

sentiment
positive    0.503155
negative    0.496845
Name: proportion, dtype: float64

In [12]:
##we can convert 'positive' -> 1
## and 'negative' -> 0  
df['sentiment'] = df['sentiment'].map({'positive' : 1,
                                       'negative' :0})

In [13]:
##We can also convert their datatype to 'int'  
df['sentiment'] = df['sentiment'].astype(int)

In [14]:
df['sentiment'].dtype  

dtype('int32')

In [15]:
df.head(3)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1


Text Preprocessing

In [16]:
##lower case all w
df['review'] =df['review'].str.lower()

In [17]:
import nltk
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

nltk.download('punkt')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to C:\Users\Siddharth
[nltk_data]     mishra\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Siddharth
[nltk_data]     mishra\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to C:\Users\Siddharth
[nltk_data]     mishra\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

In [18]:
import re
import nltk  
from bs4 import BeautifulSoup  
from nltk.corpus import stopwords

stop_words = set(stopwords.words('english'))

lemmatizer = WordNetLemmatizer()

def clean_tokenize_lemmatize(text):
    if not isinstance(text, str):
        return []

    # Clean text (reuse your existing steps)
    text = re.sub(r'(http|https|ftp|ssh)://([\w_-]+(?:(?:\.[\w_-]+)+))([\w.,@?^=%&:/~+#-]*[\w@?^=%&/~+#-])?', '', text)
    text = BeautifulSoup(text, 'html.parser').get_text()
    text = re.sub('[^a-zA-Z0-9]', ' ', text)
    text = " ".join([word for word in text.split() if word.lower() not in stop_words])
    text = " ".join(text.split())

    # Tokenize and lemmatize
    tokens = word_tokenize(text.lower())
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens]
    return lemmatized

In [19]:
df['token'] = df['review'].apply(clean_tokenize_lemmatize)

In [ ]:
df.head()    ##We got a new column 'token' containing the cleaned,tokenized and lemmatised words ready for 'Word Embedding'

,review,sentiment,token
0,one of the other reviewers has mentioned that ...,1,"[one, reviewer, mentioned, watching, 1, oz, ep..."
1,a wonderful little production. <br /><br />the...,1,"[wonderful, little, production, filming, techn..."
2,i thought this was a wonderful way to spend ti...,1,"[thought, wonderful, way, spend, time, hot, su..."
3,basically there's a family where a little boy ...,0,"[basically, family, little, boy, jake, think, ..."
4,"petter mattei's ""love in the time of money"" is...",1,"[petter, mattei, love, time, money, visually, ..."


Word2Vec initalisation

In [21]:
import gensim  
from gensim.models import Word2Vec

In [ ]:
model_wv = Word2Vec(sentences=df['token'],vector_size=100 ,window=5 ,min_count=1 ,sg =0)   ##each word will be converted to a vector of 100 dimensions

In [23]:
##To view the vocabulary
model_wv.wv.index_to_key

['movie',
 'film',
 'one',
 'like',
 'time',
 'good',
 'character',
 'would',
 'story',
 'get',
 'even',
 'make',
 'see',
 'really',
 'well',
 'scene',
 'much',
 'bad',
 'great',
 'show',
 'people',
 'first',
 'also',
 'way',
 'made',
 'thing',
 'life',
 'think',
 'go',
 'could',
 'know',
 'watch',
 'love',
 'never',
 'actor',
 'plot',
 'year',
 'little',
 'seen',
 'many',
 'two',
 'look',
 'say',
 'acting',
 'best',
 'end',
 'ever',
 'come',
 'better',
 'take',
 'man',
 'still',
 'work',
 'something',
 'director',
 'lot',
 'real',
 'part',
 'want',
 'give',
 'back',
 'performance',
 'find',
 'woman',
 'watching',
 'guy',
 'though',
 '10',
 'old',
 'funny',
 'play',
 'actually',
 'u',
 'going',
 'nothing',
 'new',
 'role',
 'another',
 'day',
 'girl',
 'every',
 'point',
 'pretty',
 'horror',
 'feel',
 'world',
 'star',
 'minute',
 'comedy',
 'around',
 'got',
 'cast',
 'big',
 'quite',
 'fact',
 'long',
 'thought',
 'young',
 'action',
 'enough',
 'right',
 'seems',
 'must',
 'line',


In [24]:
##Let's find the percentage similarity between two words  
model_wv.wv.similarity('film','movie')

0.8499408

In [25]:
##FInding words which are similar to a given words  
model_wv.wv.most_similar('genius')

[('ingredient', 0.9690165519714355),
 ('tarantino', 0.9688372015953064),
 ('spotlight', 0.9678300023078918),
 ('implies', 0.9674134850502014),
 ('equal', 0.9654985666275024),
 ('sophistication', 0.9652705192565918),
 ('spanish', 0.9652622938156128),
 ('richly', 0.9652550220489502),
 ('ambitious', 0.9636531472206116),
 ('beatles', 0.9629082083702087)]

Average Word2Vec or Sentence Embedding

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 9983 entries, 0 to 9999
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     9983 non-null   object
 1   sentiment  9983 non-null   int32 
 2   token      9983 non-null   object
dtypes: int32(1), object(2)
memory usage: 273.0+ KB


In [28]:
import numpy as np

def avg_word2vec(tokens, model, vector_size):
    valid_tokens = [token for token in tokens if token in model.wv]
    if not valid_tokens:
        return np.zeros(vector_size)
    return np.mean(model.wv[valid_tokens], axis=0)

# Apply to dataset
X = np.array([avg_word2vec(tokens, model_wv, 100) for tokens in df['token']])
y = df['sentiment'].values


Train-Test Split

In [29]:
from sklearn.model_selection import train_test_split  
X_train , X_test , y_train ,y_test = train_test_split(X ,y,test_size=0.25 ,random_state=42)

In [30]:
X_train.shape ,X_test.shape ,y_train.shape ,y_test.shape

((7487, 100), (2496, 100), (7487,), (2496,))

Model training 

In [31]:
from sklearn.metrics import confusion_matrix ,classification_report,accuracy_score

In [ ]:
from sklearn.linear_model import LogisticRegression  
model = LogisticRegression(max_iter=1000)

model.fit(X_train ,y_train)  
y_pred_wv = model.predict(X_test)  


print(accuracy_score(y_test,y_pred_wv))
print(confusion_matrix(y_test,y_pred_wv))
print(classification_report(y_test,y_pred_wv)) 





   

0.8100961538461539
[[1005  231]
 [ 243 1017]]
              precision    recall  f1-score   support

           0       0.81      0.81      0.81      1236
           1       0.81      0.81      0.81      1260

    accuracy                           0.81      2496
   macro avg       0.81      0.81      0.81      2496
weighted avg       0.81      0.81      0.81      2496



Overall our model has performed well